# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('HyunCruise').getOrCreate()

### Load Data

In [12]:
df = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [13]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [14]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



### Index Data

In [15]:
from pyspark.ml.feature import StringIndexer

In [19]:
stringIndexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_num", stringOrderType="alphabetAsc")

In [20]:
model_indexer = stringIndexer.fit(df)

In [22]:
df_indexed = model_indexer.transform(df)

In [23]:
df_indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_num|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|            0.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|            0.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|            1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|            1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|            1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|

### Create spark mlib format

In [25]:
from pyspark.ml.feature import VectorAssembler

In [30]:
model_assembler = VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers', 'cabins', 'passenger_density', 'Cruise_line_num'],
                                    outputCol='features')

In [31]:
df_assembled = model_assembler.transform(df_indexed)

In [32]:
df_assembled.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_line_num: double (nullable = false)
 |-- features: vector (nullable = true)



In [33]:
df_assembled.select('features').show()

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,30.276999999...|
|[26.0,47.262,14.8...|
|[11.0,110.0,29.74...|
|[17.0,101.353,26....|
|[22.0,70.367,20.5...|
|[15.0,70.367,20.5...|
|[23.0,70.367,20.5...|
|[19.0,70.367,20.5...|
|[6.0,110.23899999...|
|[10.0,110.0,29.74...|
|[28.0,46.052,14.5...|
|[18.0,70.367,20.5...|
|[17.0,70.367,20.5...|
|[11.0,86.0,21.24,...|
|[8.0,110.0,29.74,...|
|[9.0,88.5,21.24,1...|
|[15.0,70.367,20.5...|
|[12.0,88.5,21.24,...|
|[20.0,70.367,20.5...|
+--------------------+
only showing top 20 rows



In [34]:
df_final = df_assembled.select('features', 'crew')

In [36]:
df_final.head(5)

[Row(features=DenseVector([6.0, 30.277, 6.94, 3.55, 42.64, 0.0]), crew=3.55),
 Row(features=DenseVector([6.0, 30.277, 6.94, 3.55, 42.64, 0.0]), crew=3.55),
 Row(features=DenseVector([26.0, 47.262, 14.86, 7.43, 31.8, 1.0]), crew=6.7),
 Row(features=DenseVector([11.0, 110.0, 29.74, 14.88, 36.99, 1.0]), crew=19.1),
 Row(features=DenseVector([17.0, 101.353, 26.42, 13.21, 38.36, 1.0]), crew=10.0)]

### Train model

In [37]:
train_data, test_data = df_final.randomSplit([0.7, 0.3])

In [39]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              110|
|   mean|7.930909090909103|
| stddev|3.506815263798766|
|    min|             0.59|
|    max|             19.1|
+-------+-----------------+



In [40]:
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               48|
|   mean|7.480833333333336|
| stddev|3.512467662201916|
|    min|              1.6|
|    max|             21.0|
+-------+-----------------+



In [41]:
from pyspark.ml.regression import LinearRegression

In [42]:
model_lr = LinearRegression(labelCol='crew')

In [43]:
fitted_model_lr = model_lr.fit(train_data)

### Test Model

In [44]:
test_results = fitted_model_lr.evaluate(test_data)

In [45]:
df_final.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [46]:
test_results.rootMeanSquaredError

0.6868238596030103

In [47]:
test_results.r2adj

0.9552365594860907

### Deploy Model

In [49]:
unlabeled_data = df_final.select('features')

In [50]:
predictions = fitted_model_lr.transform(unlabeled_data)

In [52]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[6.0,30.276999999...| 4.487848347691593|
|[6.0,30.276999999...| 4.487848347691593|
|[26.0,47.262,14.8...| 6.833405441246418|
|[11.0,110.0,29.74...|12.697177636195855|
|[17.0,101.353,26....|11.576003426602272|
|[22.0,70.367,20.5...|  9.00557886184715|
|[15.0,70.367,20.5...|  8.99421512384851|
|[23.0,70.367,20.5...| 9.017161437008257|
|[19.0,70.367,20.5...| 9.000708688419161|
|[6.0,110.23899999...|11.648309891477181|
|[10.0,110.0,29.74...|12.687513911617172|
|[28.0,46.052,14.5...|6.7088846092211725|
|[18.0,70.367,20.5...| 8.999085297276498|
|[17.0,70.367,20.5...| 8.997461906133836|
|[11.0,86.0,21.24,...| 9.765565595479385|
|[8.0,110.0,29.74,...|12.684267129331847|
|[9.0,88.5,21.24,1...| 9.850636271299392|
|[15.0,70.367,20.5...|  8.99421512384851|
|[12.0,88.5,21.24,...|10.659539788329178|
|[20.0,70.367,20.5...| 9.002332079561823|
+--------------------+------------